In [2]:
import torch
import torch.nn.functional as F
import math
import numpy as np
import torch.nn as nn

# 研究pytorch中的 device 已经 GPU 的利用
## pytorch中可以通过tensor.to(device)，将这个 tensor 的后续计算放到指定的 device 上进行，但是一个 device 有多个核心，这些核心又如何实现效率最大化呢？


In [ ]:
# 测试cpu的多个核心的并行策略
# 设置使用的线程数
torch.set_num_threads(4)  # 使用4个CPU核心，从电脑的activity monitor看，不止 4 个在工作，似乎这个参数不是和核心一一对应的，


# 这些运算会自动并行化
a = torch.randn(10000, 10000)
b = torch.randn(10000, 10000)


for _ in range(1000000):

    # 矩阵乘法 - 自动分配到多个核心
    c = torch.mm(a, b)  # 并行执行！

    # 元素级运算 - 自动并行
    d = a + b           # 并行！
    e = torch.relu(a)   # 并行！
    f = a * b           # 并行！
# 注意：实际的并行效率取决于任务的大小和系统的负载情况。    
# 输出cpu各个核心的负载情况需要借助系统工具，Python本身无法直接获取。


KeyboardInterrupt: 

PyTorch Python API (你写的代码)
    ↓
PyTorch C++ 后端 (ATen)
    ↓
计算后端: CUDA / MPS / CPU (OpenMP, MKL等)
    ↓
硬件: GPU流处理器 / CPU多核心

"""
    PyTorch框架层面的"内置并行"：
    
    指的是PyTorch在框架设计时，就为各种运算设计了
    自动并行执行的能力，无论后端是什么硬件
    """
    
    # 这些运算在PyTorch框架层面就是并行友好的
    parallel_operations = {
        'element_wise': ['torch.add', 'torch.mul', 'torch.relu'],
        'matrix_ops': ['torch.matmul', 'torch.mm', 'torch.bmm'],
        'reductions': ['torch.sum', 'torch.mean', 'torch.max'],
        'convolution': ['torch.conv1d', 'torch.conv2d', 'torch.conv3d'],
        'pooling': ['torch.max_pool2d', 'torch.avg_pool2d']
    }
"""

数据并行（Data Parallelism）和张量并行（Tensor Parallelism）是两种不同的并行策略。你的理解基本正确，下面我来详细解释一下，并回答拆、算、合这三个动作是由谁支持的。

数据并行（Data Parallelism）
概念：将训练数据分成多个小批量，每个GPU上复制相同的模型，然后每个GPU处理一个小批量。每个GPU独立计算梯度，然后通过集合通信（如All-Reduce）来同步梯度。

支持：在PyTorch中，可以通过torch.nn.DataParallel（单机多卡）或torch.nn.parallel.DistributedDataParallel（单机或多机多卡）来实现。

张量并行（Tensor Parallelism）
概念：将模型中的张量（参数）拆分成多个部分，分布到不同的GPU上。每个GPU只拥有张量的一部分，然后通过通信来合并结果。例如，一个大的矩阵乘法可以拆分成多个小矩阵乘法，然后在多个GPU上执行。

支持：张量并行通常需要模型本身的支持，即需要将模型中的线性层、注意力层等设计为可拆分的。目前PyTorch本身没有内置的张量并行功能，但是有一些第三方库实现了张量并行，例如：

Megatron-LM（由NVIDIA开发，用于训练大型Transformer模型）

PyTorch的FSDP（完全分片数据并行）虽然主要是模型并行，但也涉及参数分片。

拆、算、合三个动作的支持
拆（Split）：通常由用户或模型并行框架来定义如何拆分张量。例如，将一个大矩阵按行或按列拆分。

算（Compute）：每个GPU上的计算由PyTorch和相应的后端（CUDA/MPS）来执行。但是，计算的方式需要根据拆分的方式调整，例如，矩阵乘法在拆分后需要相应的局部矩阵乘法和通信来合并结果。

合（Merge）：通过集合通信（如All-Reduce、All-Gather等）来合并结果。这通常由通信库（如NCCL）支持，PyTorch的分布式模块（torch.distributed）提供了这些通信原语。

总结
数据并行：PyTorch内置支持（DataParallel和DistributedDataParallel）。

张量并行：目前PyTorch没有直接内置，需要借助第三方库或自己实现。但是PyTorch提供了分布式通信原语（如torch.distributed）和自定义模型拆分的工具，使得实现张量并行成为可能。

因此，拆、算、合这三个动作中，拆和合需要由用户或第三方库来设计实现，而算则是由PyTorch和后端（CUDA/MPS）来执行。通信部分由PyTorch的分布式模块和底层的通信库（如NCCL）支持


In [3]:
scale = torch.sqrt(torch.FloatTensor([10]))
scale

tensor([3.1623])